In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

To analyze the impact of PrEP approval on HIV prevalence, we will conduct a Difference-in-Differences (DiD) analysis.

1. Research Question
Did the introduction of PrEP lead to a significant reduction in HIV prevalence in developed countries?

Methodology

A. Define the Treatment and Control Groups

Treatment Group: Countries that approved PrEP in a given year.

Control Group: Countries that had not yet approved PrEP by that year.


Treatment Group: Canada, Australia, France, Germany, Norway, Switzerland, Denmark, UK.

Control Group: Countries that approved PrEP after 2016 (e.g., Belgium, Ireland, Italy, Spain, Poland).

B. Difference-in-Differences (DiD) Approach

Baseline Period: 5 years before PrEP approval.

Post-Treatment Period: 5 years after PrEP approval.

Assumption: Without PrEP, the treatment and control groups would have had similar trends in HIV prevalence.

3. Data Preparation
4. 
Convert the dataset into a panel format: Country | Year | HIV Prevalence | PrEP Approval Year.

Add a "Post-PrEP" column:

1 if year is after PrEP approval.

0 if year is before PrEP approval.

Label countries as treatment or control based on approval year.

6. Visualization
   
Trend Line Graphs:

Plot HIV prevalence before and after PrEP introduction.

Compare treatment and control countries.

DiD Visualization:

Show the difference in HIV prevalence trends pre- and post-PrEP.

8. Hypothesis Testing
Null Hypothesis (
​
 ): PrEP has no effect on HIV prevalence.
Alternative Hypothesis (
𝐻
𝐴
H 
A
​
 ): PrEP reduces HIV prevalence.

Run linear regression with fixed effects to control for country-specific factors.

10. Expected Findings
    
If PrEP was effective, we should see a greater decline in HIV prevalence in treatment countries compared to control countries.

If no significant effect is found, other factors (e.g., testing expansion, ART improvements) might be driving HIV prevalence trends.



In [ ]:
import pandas as pd
HIV = pd.read_csv("/kaggle/input/hiv-prevalence/updated 2.csv")

In [ ]:
HIV.head()

Reshape the Data from Wide to Long Format

We’ll first load the dataset, then reshape it into long format.

In [ ]:
import pandas as pd

# Load your data (replace with the actual path)
HIV = pd.read_csv('/kaggle/input/hiv-prevalence/updated 2.csv')

# Reshape from wide to long format
data_long = pd.melt(HIV, 
                    id_vars=['Country Name', 'Country Code', 'Series Name', 'Series Code'], 
                    var_name='Year', 
                    value_name='HIV_Incidence')

# Convert 'Year' to integer for comparison
data_long['Year'] = data_long['Year'].str.extract('(\d+)').astype(int)

# View the reshaped data
data_long.head()


Merge PrEP Approval Years

Next, create a PrEP approval year dataset.

Country	PrEP   Approval Year	
United States  2012	
Canada	       2016	
Australia	   2016	
France	       2015	
Germany	       2016	
Norway	       2016	
United Kingdom 2016	
Belgium	       2017	
Ireland	       2017	
Italy	       2017	
Netherlands	   2017	
New Zealand	   2018	
Sweden	       2017	
Switzerland	   2016	
Denmark    	   2016	
Greece	       2017	
Luxembourg	   2017	
Portugal	   2017	
Austria	       2017	
Spain	       2019	
Poland	       2019	
Japan	   Not Approved	

In [ ]:
# Sample PrEP approval data (replace this with actual data for each country)
prep_data = {
    'Country Name': ['Canada', 'United States', 'Australia', 'Germany', 'United Kingdom','France','Norway','Belgium','Ireland','Italy','Netherlands','Newzealand','Sweden','Switzerland','Denmark','Greece','Luxembourg','Portugal','Austria','Spain','Poland','Japan'],
    'PrEP_Approval_Year': [2016, 2012, 2016, 2016, 2016,2015,2016,2017,2017,2017,2017,2018,2017,2016,2016,2017,2017,2017,2017,2019,2019,None]  # Example approval years
}

prep_df = pd.DataFrame(prep_data)

# Merge the PrEP approval year with the long-format HIV data
data_long = pd.merge(data_long, prep_df, on='Country Name', how='left')

# View the data with merged PrEP approval year
data_long.head()


In [ ]:
# Convert 'PrEP_Approval_Year' to integer while handling NaNs correctly
data_long['PrEP_Approval_Year'] = pd.to_numeric(data_long['PrEP_Approval_Year'], errors='coerce')

# Now convert the column to nullable Int64 type, which can handle NaN correctly
data_long['PrEP_Approval_Year'] = data_long['PrEP_Approval_Year'].astype('Int64')

# Check the result
data_long.head()


Create Treatment Indicators

Now, we need to create two indicators:

Post_PrEP: 1 if year > PrEP approval year, 0 otherwise.

Treatment: 1 if the country has PrEP approval, 0 otherwise.

DiD_Term: Interaction of Post_PrEP and Treatment.



In [ ]:
# Create the Post_PrEP indicator (1 if year > PrEP approval year, else 0)
data_long['Post_PrEP'] = (data_long['Year'] > data_long['PrEP_Approval_Year']).astype('Int64')

# Create the Treatment indicator (1 if country has PrEP, 0 otherwise)
data_long['Treatment'] = data_long['PrEP_Approval_Year'].notna().astype('Int64')

# Create the DiD Term (interaction of Post_PrEP and Treatment)
data_long['DiD_Term'] = data_long['Post_PrEP'] * data_long['Treatment']

# Check the dataset with the newly created indicators
data_long.head()


Run Difference-in-Differences (DiD) Regression

We will run the DiD regression for each age group separately (All Ages, Ages 15-49, and Ages 15-24

In [ ]:
# Check for missing values in relevant columns
missing_data = data_long[['HIV_Incidence', 'Post_PrEP', 'Treatment', 'DiD_Term', 'Country Name', 'Year']].isnull().sum()
print("Missing values in each column:\n", missing_data)


Fill missing values with the mean (or median)

In [ ]:
#Round the mean to an integer before filling
# Fill missing values with rounded mean (for integer columns)
data_long['Post_PrEP'] = data_long['Post_PrEP'].fillna(round(data_long['Post_PrEP'].mean()))
data_long['DiD_Term'] = data_long['DiD_Term'].fillna(round(data_long['DiD_Term'].mean()))

# Check if missing data has been handled
missing_data_cleaned = data_long[['HIV_Incidence', 'Post_PrEP', 'Treatment', 'DiD_Term', 'Country Name', 'Year']].isnull().sum()
print("Missing values after filling:\n", missing_data_cleaned)


In [ ]:
data_long.head()

In [ ]:
import statsmodels.formula.api as smf
import pandas as pd

# Rename columns to avoid spaces
data_long = data_long.rename(columns={'Country Name': 'Country_Name'})

# Filter data for the three series (age groups)
age_groups = [
    'Incidence of HIV, all (per 1,000 uninfected population)', 
    'Incidence of HIV, ages 15-49 (per 1,000 uninfected population ages 15-49)', 
    'Incidence of HIV, ages 15-24 (per 1,000 uninfected population ages 15-24)'
]

# Initialize dictionary to store results
results = {}

for group in age_groups:
    # Filter the data for the current age group
    group_data = data_long[data_long['Series Name'] == group]
    
    # Ensure the 'HIV_Incidence' column is not a string (convert if necessary)
    group_data['HIV_Incidence'] = pd.to_numeric(group_data['HIV_Incidence'], errors='coerce')

    # DiD regression model
    model = smf.ols('HIV_Incidence ~ Post_PrEP + Treatment + DiD_Term + C(Country_Name) + C(Year)', 
                    data=group_data).fit()

    # Store the results
    results[group] = model.summary()

# View the DiD regression results
for group, result in results.items():
    print(f"Results for {group}:\n")
    print(result)
    print("\n" + "="*50 + "\n")


Checking Robustness for the Difference-in-Differences (DiD) Model

To ensure your DiD regression results are reliable, you should apply various robustness checks. Below are key methods along with their code implementations.

Use Robust Standard Errors

The standard OLS errors may be biased due to heteroskedasticity or autocorrelation.

Use Heteroskedasticity and Autocorrelation Consistent (HAC) robust errors.

In [ ]:
for group in age_groups:
    group_data = data_long[data_long['Series Name'] == group]
    group_data['HIV_Incidence'] = pd.to_numeric(group_data['HIV_Incidence'], errors='coerce')

    # OLS with robust standard errors
    model = smf.ols('HIV_Incidence ~ Post_PrEP + Treatment + DiD_Term + C(Country_Name) + C(Year)', 
                    data=group_data).fit(cov_type='HC3')  # HC3 gives heteroskedasticity-robust SEs

    results[group] = model.summary()

for group, result in results.items():
    print(f"Results for {group} (with robust SEs):\n")
    print(result)
    print("\n" + "="*50 + "\n")


Placebo Test (Falsification Test)

Assign a fake treatment year before PrEP was actually introduced.

If the effect is significant before the intervention, there may be a problem.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter data for the years 2000-2022
filtered_data = data_long[(data_long['Year'] >= 2000) & (data_long['Year'] <= 2022)]

# Get unique country names
countries = filtered_data['Country_Name'].unique()

# Define color palette
palette = {'Control': 'blue', 'Treatment': 'red'}

for country in countries:
    # Filter data for the current country
    country_data = filtered_data[filtered_data['Country_Name'] == country]
    
    plt.figure(figsize=(10, 6))  # Create a new figure for each plot

    # Define Treatment & Control labels
    country_data['Group'] = country_data['Treatment'].map({0: 'Control', 1: 'Treatment'})

    # Line plot
    sns.lineplot(
        data=country_data, x='Year', y='HIV_Incidence', hue='Group', 
        style='Group', markers=True, dashes=False, palette=palette, 
        linewidth=2.5, markersize=8
    )

    # Titles and labels
    plt.title(f"Difference-in-Differences: {country} (2000-2022)", fontsize=14)
    plt.xlabel("Year", fontsize=12)
    plt.ylabel("HIV Incidence", fontsize=12)
    plt.xticks(fontsize=11)
    plt.yticks(fontsize=11)

    # Add vertical line for PrEP introduction
    min_treated_year = country_data[country_data['Post_PrEP'] == 1]['Year'].min()
    if not pd.isna(min_treated_year):
        plt.axvline(x=min_treated_year, color='black', linestyle="--", lw=2, label="PrEP Introduction")

    # Show legend and plot
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter data for Canada only
canada_data = data_long[data_long["Country_Name"] == "Canada"]

# Define age groups
age_groups = {
    "All Ages": "Incidence of HIV, all (per 1,000 uninfected population)",
    "Ages 15-49": "Incidence of HIV, ages 15-49 (per 1,000 uninfected population ages 15-49)",
    "Ages 15-24": "Incidence of HIV, ages 15-24 (per 1,000 uninfected population ages 15-24)"
}

# Set figure size
fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

# Plot for each age group
for ax, (title, series_name) in zip(axes, age_groups.items()):
    subset = canada_data[canada_data["Series Name"] == series_name]  # Filter for the age group
    sns.lineplot(x="Year", y="HIV_Incidence", data=subset, ax=ax, marker="o", color="b")
    
    ax.set_title(f"HIV Incidence in Canada: {title}", fontsize=14)
    ax.set_ylabel("HIV Incidence Rate")
    ax.grid(True)

# Common X-axis label
plt.xlabel("Year", fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
